## Install required Dependincies

In [1]:
! pip install num2words
! pip install datasets

## import library


In [2]:
import json,datasets,num2words
import os,librosa,re
import pandas as pd
from datasets import Dataset, DatasetDict,load_from_disk


/home/sumitkrchoubey/anaconda3/envs/audio_text/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## read the dataset file which create using listin audio files


In [3]:
open_dataset = open('data_set_create2.json')
data = json.load(open_dataset)

## create code for data_set creation

In [7]:
def create_dataset_format_with_convert_number_to_words(text_data):
    data_set_create={}
    check_hash_in_audio_link="/"
    check_dot_in_audio_link="."
    audio_file_path="chunk"+"/"
    save_audio_file_path_model_training="/content/drive/MyDrive/chunk"+"/"
    if check_hash_in_audio_link in text_data['audio_link']:
        audio_link=text_data['audio_link'].split(check_hash_in_audio_link)[-1]
        audio_text=text_data['audio_text']
        clean_text_data=re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), audio_text)
        #clean_text_data
        #print(clean_text_data,audio_link)
        first_file_array,first_file_sample_rate=librosa.load(audio_file_path+audio_link,sr=16_000)
        data_set_create['audio']={"array":(first_file_array),"path":save_audio_file_path_model_training+audio_link,"sampling_rate":first_file_sample_rate}
        data_set_create['file']=save_audio_file_path_model_training+audio_link
        data_set_create['text']=clean_text_data
    elif check_dot_in_audio_link in text_data['audio_link']:
        audio_link=text_data['audio_link']
        audio_text=text_data['audio_text']
        clean_text_data=re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), audio_text)
        
        #print(clean_text_data,audio_link)
        first_file_array,first_file_sample_rate=librosa.load(audio_file_path+audio_link,sr=16_000)
        data_set_create['audio']={"array":(first_file_array),"path":save_audio_file_path_model_training+audio_link,"sampling_rate":first_file_sample_rate}
        data_set_create['file']=save_audio_file_path_model_training+audio_link
        data_set_create['text']=clean_text_data
        #data_set.append(data_set_create)
    else:
        audio_link=text_data['audio_link']+".wav"
        audio_text=text_data['audio_text']
        clean_text_data=re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), audio_text)
        #print(clean_text_data,audio_link)
        
        first_file_array,first_file_sample_rate=librosa.load(audio_file_path+audio_link,sr=16_000)
        data_set_create['audio']={"array":(first_file_array),"path":save_audio_file_path_model_training+audio_link,"sampling_rate":first_file_sample_rate}
        data_set_create['file']=save_audio_file_path_model_training+audio_link
        data_set_create['text']=clean_text_data
    return data_set_create

## Create function to run dataset create function  and save the output into the huggface dataset format

# ## Three dataset_types

## base data_set=data_set_value
## large data_set=increase_data_set
## small_data_set=sample_size

In [8]:
def processing_dataset(data):
    ## len of list of dictionary
    len_list_dicionary=len(data)
    i = 0
    data_set=[]
    while i < len_list_dicionary:
        d=(create_dataset_format_with_convert_number_to_words(data[i]))
        data_set.append(d)
        i += 1
    ## increase the dataset
    copy_times=5
    increase_data_set=data_set*copy_times
    ## select 200 sample_size_of from base data
    sample_size=data_set[:200]
    ## save basemodel
    create_dataset_huggface_base_model=datasets.Dataset.from_pandas(pd.DataFrame(data=data_set))
    create_dataset_huggface_base_model.save_to_disk('call_recording_data_set_base')
    ## save largemodel
    create_dataset_huggface_large_model=datasets.Dataset.from_pandas(pd.DataFrame(data=increase_data_set))
    create_dataset_huggface_large_model.save_to_disk('call_recording_data_set_large')
    ## save smallmodel
    create_dataset_huggface_small_model=datasets.Dataset.from_pandas(pd.DataFrame(data=sample_size))
    create_dataset_huggface_small_model.save_to_disk('call_recording_data_set_small')
    return "done"

In [9]:
data_set_value=processing_dataset(data)